In [1]:
import pandas as pd
from pulp import *

# Brewery Transport
## https://coin-or.github.io/pulp/CaseStudies/a_transportation_problem.html
A boutique brewery has two warehouses from which it distributes beer to five carefully chosen bars. At the start of every week, each bar sends an order to the brewery’s head office for so many crates of beer, which is then dispatched from the appropriate warehouse to the bar. The brewery would like to have an interactive computer program which they can run week by week to tell them which warehouse should supply which bar so as to minimize the costs of the whole operation. For example, suppose that at the start of a given week the brewery has 1000 cases at warehouse A, and 4000 cases at warehouse B, and that the bars require 500, 900, 1800, 200, and 700 cases respectively. Which warehouse should supply which bar?

In [40]:
demand = {'1': 500, '2': 900, '3': 1800, '4':200, '5':700}

In [41]:
resource = {'A': 1000, 'B': 4000}

In [42]:
model = LpProblem('Transport_Cost_Minimisation', LpMinimize)

In [43]:
num_trans = LpVariable.dicts('num', [(i,j) for i in resource.keys() for j in demand.keys()],lowBound=0, cat = 'Integer')

In [61]:
# route_active = LpVariable.dicts('active', [(i,j) for i in resource.keys() for j in demand.keys()], cat = 'Binary')

In [45]:
costs = pd.DataFrame(index = ['A', 'B'], columns = ['1', '2', '3', '4', '5'])

In [46]:
costs['1'] = [2,3]
costs['2'] = [4,1]
costs['3'] = [5,3]
costs['4'] = [2,2]
costs['5'] = [1,3]

In [47]:
costs

,1,2,3,4,5
A,2,4,5,2,1
B,3,1,3,2,3


In [48]:
# Objective: fulfil orders at lowest cost
model += lpSum([num_trans[(i,j)]*costs.loc[i,j] for i in resource.keys() for j in demand.keys()])

In [49]:
# Constraint that demand is met
for j in demand.keys():
    model += lpSum([num_trans[(i,j)] for i in resource.keys()]) == demand[j], f"Demand at {j}"

In [54]:
# Constraint that no warehouse supplies too much
for i in resource.keys():
    model += lpSum([num_trans[(i,j)] for j in demand.keys()]) <= resource[i], f"Limit from {i}"

In [55]:
model.solve()

1

In [56]:
print(f"Model Status: {LpStatus[model.status]}")

Model Status: Optimal


In [57]:
print(f"Objective Value: {model.objective.value()}")

Objective Value: 8600.0


In [58]:
for v in model.variables():
    print(v.name,v.varValue)

num_('A',_'1') 300.0
num_('A',_'2') 0.0
num_('A',_'3') 0.0
num_('A',_'4') 0.0
num_('A',_'5') 700.0
num_('B',_'1') 200.0
num_('B',_'2') 900.0
num_('B',_'3') 1800.0
num_('B',_'4') 200.0
num_('B',_'5') 0.0


In [59]:
for name, c in model.constraints.items():
    print(name, c.pi, c.slack)

Demand_at_1 0.0 -0.0
Demand_at_2 0.0 -0.0
Demand_at_3 0.0 -0.0
Demand_at_4 0.0 -0.0
Demand_at_5 0.0 -0.0
Limit_from_A 0.0 -0.0
Limit_from_B 0.0 900.0
